In [99]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import os

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

#Evalaluation
from sklearn.cross_validation import cross_val_score
from sklearn import metrics

# Grid
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV

#Performance
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


## Load

In [113]:
X.head()

,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,3,0,1,0,0,1.0,0
1,1,1,2,3,1,3.0,0
2,3,1,1,1,0,2.0,1
3,1,1,2,3,0,3.0,0
4,3,0,2,1,0,1.0,1


In [ ]:
os.chdir(r"D:/My Computer/DATA/Udacity")
os.getcwd()

In [122]:
#Titanic
path = r"C:/Users/Nicol/Google Drive/Learning/Jupyter/Titanic"
train_df = pd.read_csv(open(os.path.join(path, "clean_train.csv"), "r")) 
test_df = pd.read_csv(open(os.path.join(path, "clean_test.csv"), "r")) 

X = train_df.drop(["Survived", "Age*Class"] , axis=1)
y = train_df["Survived"]

oosample  = test_df.drop(["PassengerId","Age*Class"] , axis=1).copy()
print(X.shape, y.shape, oosample.shape)

def submit(model, name):
    model.fit(X, y)
    submission = model.predict(oosample)

    df = pd.DataFrame({'PassengerId':test_df.PassengerId, 
                       'Survived':submission})

    print(len(df))
    df.to_csv(name,header=True,index=False)

(891, 7) (891,) (418, 7)


In [116]:
def evalsingle(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(metrics.accuracy_score(y_test, y_pred))

In [98]:
# use train/test split with different random_state values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((668, 8), (668,), (223, 8), (223,))

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Pclass       891 non-null int64
Sex          891 non-null int64
Age          891 non-null int64
Fare         891 non-null int64
Embarked     891 non-null int64
Title        891 non-null float64
IsAlone      891 non-null int64
Age*Class    891 non-null int64
dtypes: float64(1), int64(7)
memory usage: 55.8 KB


In [9]:
import seaborn as sns; sns.set()
#sns.pairplot(train_df, hue='Survived', size=1.5);

# Generative Classification
Probabilistically determine the label for a new point

## Gaussian

In [6]:
%%time
%%memit
model = GaussianNB()
model.fit(X_train, y_train)
y_pred= model.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred)*100)
print(cross_val_score(model, X, y, cv=10, scoring='accuracy').mean()*100)

74.8878923767
72.5175632732
74.8878923767
72.5175632732
74.8878923767
72.5175632732
peak memory: 130.41 MiB, increment: 0.44 MiB
Wall time: 1.91 s


## Logistic


In [117]:
%%time
%%memit
model= LogisticRegression()

#Fit Model
scores= cross_val_score(model, X, y, cv=10, scoring='accuracy')
print(scores.mean()*100)

80.8173589831
80.8173589831
peak memory: 122.46 MiB, increment: 0.09 MiB
Wall time: 1.11 s


# Non-Parametric

# Esemble Method
## Bagging

https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/
HyperParameters:
- max_features: limits number of number of features in a tree
- n_estimators: # of trees built before average prediciton is made
- min_sample_leaf: End node of trees. Too small = more noise. For Regression tree.
- n_jobs: computer processors utilized. -1 = no restrictions
- random_state: seed()

In [23]:
tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=300, max_samples=0.8,
                        random_state=1)

print(cross_val_score(bag, X, y, cv=10, scoring='accuracy').mean()*100)

81.0445749631


In [145]:
%%time
%%memit
param_grid ={'max_features': [.2,.3,.4, .5,.6,.8, 1],
             'n_estimators': [40,50,60,80,100,120,150,180],
             'max_samples': [0.2,0.3,.4,.5,.6]}

tree = DecisionTreeClassifier()
#bag = BaggingClassifier(tree)

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(BaggingClassifier(tree, n_jobs=1),
                    param_grid, cv=cv, scoring='accuracy',
                    verbose=1)

grid.fit(X, y);
print(grid.best_score_*100)
print(grid.best_params_)
print(grid.best_estimator_)
# model = grid.best_estimator_

Fitting 5 folds for each of 280 candidates, totalling 1400 fits
[CV] max_samples=0.2, n_estimators=40, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=40, max_features=0.2, score=0.748603, total=   0.0s
[CV] max_samples=0.2, n_estimators=40, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=40, max_features=0.2, score=0.715084, total=   0.0s
[CV] max_samples=0.2, n_estimators=40, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=40, max_features=0.2, score=0.659218, total=   0.0s
[CV] max_samples=0.2, n_estimators=40, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=40, max_features=0.2, score=0.743017, total=   0.0s
[CV] max_samples=0.2, n_estimators=40, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=40, max_features=0.2, score=0.709497, total=   0.0s
[CV] max_samples=0.2, n_estimators=50, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=50, max_features=0.2, score=0.6815

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s


[CV]  max_samples=0.2, n_estimators=50, max_features=0.2, score=0.743017, total=   0.0s
[CV] max_samples=0.2, n_estimators=50, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=50, max_features=0.2, score=0.709497, total=   0.0s
[CV] max_samples=0.2, n_estimators=50, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=50, max_features=0.2, score=0.687151, total=   0.0s
[CV] max_samples=0.2, n_estimators=50, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=50, max_features=0.2, score=0.731844, total=   0.0s
[CV] max_samples=0.2, n_estimators=60, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=60, max_features=0.2, score=0.720670, total=   0.0s
[CV] max_samples=0.2, n_estimators=60, max_features=0.2 ..............


[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.2s remaining:    0.0s


[CV]  max_samples=0.2, n_estimators=60, max_features=0.2, score=0.737430, total=   0.0s
[CV] max_samples=0.2, n_estimators=60, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=60, max_features=0.2, score=0.759777, total=   0.0s
[CV] max_samples=0.2, n_estimators=60, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=60, max_features=0.2, score=0.698324, total=   0.0s
[CV] max_samples=0.2, n_estimators=60, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=60, max_features=0.2, score=0.731844, total=   0.0s
[CV] max_samples=0.2, n_estimators=80, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=80, max_features=0.2, score=0.720670, total=   0.0s
[CV] max_samples=0.2, n_estimators=80, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=80, max_features=0.2, score=0.748603, total=   0.0s
[CV] max_samples=0.2, n_estimators=80, max_features=0.2 ..............
[CV]  max_samples=0.2, n_estimators=80, max_fe

...

[CV]  max_samples=0.2, n_estimators=180, max_features=1, score=0.759777, total=   0.0s
[CV] max_samples=0.2, n_estimators=180, max_features=1 ...............
[CV]  max_samples=0.2, n_estimators=180, max_features=1, score=0.726257, total=   0.0s
[CV] max_samples=0.2, n_estimators=180, max_features=1 ...............
[CV]  max_samples=0.2, n_estimators=180, max_features=1, score=0.743017, total=   0.0s
[CV] max_samples=0.2, n_estimators=180, max_features=1 ...............
[CV]  max_samples=0.2, n_estimators=180, max_features=1, score=0.731844, total=   0.0s
[CV] max_samples=0.3, n_estimators=40, max_features=1 ................
[CV]  max_samples=0.3, n_estimators=40, max_features=1, score=0.759777, total=   0.0s
[CV] max_samples=0.3, n_estimators=40, max_features=1 ................
[CV]  max_samples=0.3, n_estimators=40, max_features=1, score=0.754190, total=   0.0s
[CV] max_samples=0.3, n_estimators=40, max_features=1 ................
[CV]  max_samples=0.3, n_estimators=40, max_features=1

[CV]  max_samples=0.4, n_estimators=60, max_features=1, score=0.743017, total=   0.0s
[CV] max_samples=0.4, n_estimators=60, max_features=1 ................
[CV]  max_samples=0.4, n_estimators=60, max_features=1, score=0.709497, total=   0.0s
[CV] max_samples=0.4, n_estimators=60, max_features=1 ................
[CV]  max_samples=0.4, n_estimators=60, max_features=1, score=0.754190, total=   0.0s
[CV] max_samples=0.4, n_estimators=60, max_features=1 ................
[CV]  max_samples=0.4, n_estimators=60, max_features=1, score=0.726257, total=   0.0s
[CV] max_samples=0.4, n_estimators=80, max_features=1 ................
[CV]  max_samples=0.4, n_estimators=80, max_features=1, score=0.692737, total=   0.0s
[CV] max_samples=0.4, n_estimators=80, max_features=1 ................
[CV]  max_samples=0.4, n_estimators=80, max_features=1, score=0.770950, total=   0.0s
[CV] max_samples=0.4, n_estimators=80, max_features=1 ................
[CV]  max_samples=0.4, n_estimators=80, max_features=1, sc

[CV]  max_samples=0.5, n_estimators=120, max_features=1, score=0.726257, total=   0.0s
[CV] max_samples=0.5, n_estimators=120, max_features=1 ...............
[CV]  max_samples=0.5, n_estimators=120, max_features=1, score=0.759777, total=   0.0s
[CV] max_samples=0.5, n_estimators=120, max_features=1 ...............
[CV]  max_samples=0.5, n_estimators=120, max_features=1, score=0.754190, total=   0.0s
[CV] max_samples=0.5, n_estimators=120, max_features=1 ...............
[CV]  max_samples=0.5, n_estimators=120, max_features=1, score=0.754190, total=   0.0s
[CV] max_samples=0.5, n_estimators=120, max_features=1 ...............
[CV]  max_samples=0.5, n_estimators=120, max_features=1, score=0.720670, total=   0.0s
[CV] max_samples=0.5, n_estimators=150, max_features=1 ...............
[CV]  max_samples=0.5, n_estimators=150, max_features=1, score=0.720670, total=   0.0s
[CV] max_samples=0.5, n_estimators=150, max_features=1 ...............
[CV]  max_samples=0.5, n_estimators=150, max_feature

[CV]  max_samples=0.6, n_estimators=180, max_features=1, score=0.731844, total=   0.0s
[CV] max_samples=0.6, n_estimators=180, max_features=1 ...............
[CV]  max_samples=0.6, n_estimators=180, max_features=1, score=0.737430, total=   0.0s
[CV] max_samples=0.6, n_estimators=180, max_features=1 ...............
[CV]  max_samples=0.6, n_estimators=180, max_features=1, score=0.726257, total=   0.1s
80.5586592179
{'max_samples': 0.4, 'n_estimators': 180, 'max_features': 0.6}
BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=0.6,
         max_samples=0.4, n_estimators=180, n_jobs=1, oob_score=False,
         random_state=None, verbos

[Parallel(n_jobs=1)]: Done 1400 out of 1400 | elapsed:  1.8min finished


In [147]:
model = grid.best_estimator_
print(grid.best_estimator_)
evalsingle(model)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=0.6,
         max_samples=0.4, n_estimators=180, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)
0.834080717489


In [ ]:
submit(model, "82bagger.csv")

## Random Forest

In [75]:
%%time
%%memit
model = RandomForestClassifier(n_estimators=500)
evalsingle(model)

0.962285714286
peak memory: 2181.68 MiB, increment: 790.49 MiB
Wall time: 1min 17s


In [80]:
RandomForestClassifier().get_params().keys()

dict_keys(['max_depth', 'min_impurity_split', 'n_estimators', 'oob_score', 'bootstrap', 'min_weight_fraction_leaf', 'max_features', 'class_weight', 'verbose', 'min_samples_split', 'random_state', 'n_jobs', 'warm_start', 'min_samples_leaf', 'criterion', 'max_leaf_nodes'])

In [142]:
%%time
%%memit
param_grid ={'n_estimators':[400,500,600],'max_features':[.7,.6,.5]}
#param_grid ={'n_estimators':[200]}
from sklearn import feature_selection

#Fit Model

#model = feature_selection.RFE(RandomForestClassifier())
model= RandomForestClassifier()

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(model,
                    param_grid, cv=cv, scoring='accuracy',
                    verbose=1)

grid.fit(X, y);
print(grid.best_score_*100)
#print(grid.best_params_)
#print(grid.best_estimator_)
# model = grid.best_estimator_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   21.8s finished


78.8826815642
peak memory: 148.05 MiB, increment: 9.64 MiB
Wall time: 22.9 s


In [143]:
model = grid.best_estimator_
print(grid.best_estimator_)
evalsingle(model)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.5, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=400, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
0.820627802691


In [144]:
submit(model, "randomforest82")

418


## Gradient Boosting Classifier

GB builds an additive model in a forward stage-wise fashion; it allows for the optimization of arbitrary differentiable loss functions.

In [44]:
?GradientBoostingClassifier

In [125]:
%%time
%%memit
param_grid ={'n_estimators':[150,200,250,275],
            'loss': ['deviance', 'exponential'],
            'learning_rate':[0.3,0.2,0.1,0.5],
            'max_depth': [2,3,4]}
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(GradientBoostingClassifier(),
                    param_grid,cv=cv, scoring='accuracy',
                    verbose=1)

grid.fit(X, y);
print(grid.best_score_*100)
print(grid.best_params_)
#print(grid.best_estimator_)
# model = grid.best_estimator_

Fitting 10 folds for each of 96 candidates, totalling 960 fits
83.7261503928
{'max_depth': 3, 'loss': 'exponential', 'learning_rate': 0.3, 'n_estimators': 200}
peak memory: 137.61 MiB, increment: 1.42 MiB
Wall time: 1min 40s


[Parallel(n_jobs=1)]: Done 960 out of 960 | elapsed:  1.7min finished


In [126]:
model = grid.best_estimator_
print(grid.best_estimator_)
evalsingle(model)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.3, loss='exponential', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=200, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)
0.843049327354


In [127]:
submit(model, "GradientBoostingClassifier_84.csv")

418


## AdaBoostClassifier

In [46]:
?AdaBoostClassifier

In [128]:
%%time
%%memit
param_grid ={'n_estimators':[150,200,250,275],
            'learning_rate':[0.3,0.5,0.65,0.8]}

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(AdaBoostClassifier(),
                    param_grid,cv=cv, scoring='accuracy',
                    verbose=1)

grid.fit(X, y);
print(grid.best_score_*100)
print(grid.best_params_)
#print(grid.best_estimator_)
# model = grid.best_estimator_

Fitting 10 folds for each of 16 candidates, totalling 160 fits
80.8080808081
{'learning_rate': 0.8, 'n_estimators': 150}
peak memory: 137.62 MiB, increment: 0.00 MiB
Wall time: 34.5 s


[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:   33.7s finished


In [129]:
model = grid.best_estimator_
print(grid.best_estimator_)
evalsingle(model)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.8, n_estimators=150, random_state=None)
0.856502242152


In [130]:
submit(model, "Ada_boost85.csv")

418


## KNN

In [51]:
?KNeighborsClassifier

In [56]:
list(range(1,20))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [59]:
%%time
%%memit
param_grid ={'n_neighbors': list(range(1,20)),
            'weights':['uniform','distance']
            }

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(KNeighborsClassifier(),
                    param_grid,cv=cv, scoring='accuracy',
                    verbose=1)

grid.fit(X, y);
print(grid.best_score_*100)
print(grid.best_params_)
#print(grid.best_estimator_)
# model = grid.best_estimator_

Fitting 10 folds for each of 38 candidates, totalling 380 fits
80.8080808081
{'n_neighbors': 15, 'weights': 'uniform'}
peak memory: 141.71 MiB, increment: 0.14 MiB
Wall time: 3.23 s


[Parallel(n_jobs=1)]: Done 380 out of 380 | elapsed:    2.5s finished


In [61]:
model = grid.best_estimator_
print(grid.best_estimator_)
evalsingle(model)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=15, p=2,
           weights='uniform')
0.825112107623


In [62]:
submit(model, "KNN.csv")

418


# Discriminative Classification
Model new points by seeing where it falls upon a divide.
Fast prediction phase, work well in high dimensional data, versatile

Costly at high quantities of data

## Support Vector Classifier
Creates a linear divide between point to classify. Maximizes the distance of the discriminatory margin.

Hyperparameters:
- C: Hardness of the margin. Higher C, less softening.


Radial Basis Function (RBF)
- Gamma: how far the influence of a single training example raches. low=far, high=close, Inverse of the radius of influence of samples selected by the model as support vectors.

In [131]:
%%time
%%memit

# Define Model
model = svm.LinearSVC()
#Fit Model
scores= cross_val_score(model, X, y, cv=10, scoring='accuracy')
print(scores.mean()*100)

#submit(svm.LinearSVC(), name="80linear_svc.csv")

80.2492339122
peak memory: 137.63 MiB, increment: 0.01 MiB
Wall time: 854 ms


In [60]:
?svm.SVC

In [132]:
%%time
%%memit
param_grid = [
  {'C': [50,100], 'kernel': ['linear']},
  {'C': [150], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}
 ]

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svm.SVC(),
                    param_grid, cv=cv,
                    scoring='accuracy', verbose=1)

grid.fit(X, y);
print(grid.best_score_*100)
print(grid.best_params_)
print(grid.best_estimator_)
# model = grid.best_estimator_

Fitting 10 folds for each of 4 candidates, totalling 40 fits
78.7878787879
{'gamma': 0.0001, 'kernel': 'rbf', 'C': 150}
SVC(C=150, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
peak memory: 137.69 MiB, increment: 0.06 MiB
Wall time: 2.44 s


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    1.7s finished


In [77]:
print(np.logspace(-2, 3, 5))
np.logspace(-9, 3, 13)

[  1.00000000e-02   1.77827941e-01   3.16227766e+00   5.62341325e+01
   1.00000000e+03]


array([  1.00000000e-09,   1.00000000e-08,   1.00000000e-07,
         1.00000000e-06,   1.00000000e-05,   1.00000000e-04,
         1.00000000e-03,   1.00000000e-02,   1.00000000e-01,
         1.00000000e+00,   1.00000000e+01,   1.00000000e+02,
         1.00000000e+03])

In [139]:
%%time
%%memit
C_range = np.logspace(2, 5, 7)
gamma_range = np.logspace(-4, 5, 1)
param_grid = dict(gamma=gamma_range, C=C_range)

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svm.SVC(),
                    param_grid, cv=cv,
                    scoring='accuracy', verbose=1)

grid.fit(X, y);
print(grid.best_score_*100)
print(grid.best_params_)
print(grid.best_estimator_)
# model = grid.best_estimator_

Fitting 5 folds for each of 7 candidates, totalling 35 fits
78.3240223464
{'gamma': 0.0001, 'C': 100000.0}
SVC(C=100000.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
peak memory: 138.41 MiB, increment: 0.00 MiB
Wall time: 2.33 s


[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    1.5s finished


In [140]:
model = grid.best_estimator_
print(grid.best_estimator_)
evalsingle(model)

SVC(C=100000.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.816143497758


In [141]:
submit(model, "SVCrbf.csv")

418


## PCA + SVC Pipeline

In [82]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

pca = PCA(n_components=6, whiten=True, random_state=42, svd_solver='randomized')
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)
# pipeline!

from sklearn.grid_search import GridSearchCV
param_grid = [{'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]},
              {'svc__C': [0.001, 0.01, 0.1, 1, 10, 50, 100, 150, 1000, 1500], 'svc__kernel': ['linear']}]
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(model, param_grid, cv=cv)

%time grid.fit(X, y)
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

Wall time: 21.9 s
0.792368125701459
{'svc__gamma': 0.0005, 'svc__C': 5}
Pipeline(steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=6, random_state=42,
  svd_solver='randomized', tol=0.0, whiten=True)), ('svc', SVC(C=5, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0005, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
